# The feedforward neural network model for learning PageRank

In [1]:
import time
from dihelpers import *

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

## Loading the dataset from numpy files

In [3]:
# Pageank as the target or label data
target_pagerank = np.load("wiki_vote_pageranks.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
target_pagerank[0]

array([0.00020539])

In [4]:
# scaling the PageRank data
target_pagerank = target_pagerank * 5000
target_pagerank[0:10]

array([[1.02697491],
       [0.25243912],
       [0.25243912],
       [1.55916255],
       [0.25243912],
       [1.63317788],
       [0.25243912],
       [2.10699831],
       [0.25243912],
       [0.25243912]])

In [5]:
target_pagerank.shape

(7115, 1)

In [6]:
# the order 5 NFDC matrix as the feature set
matrix_1 = np.load("wiki_vote_NDFC_matrix_sp1_rad5_inward_discounted.npy", mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

In [7]:
matrix_1.shape

(7115, 6, 15)

In [8]:
matrix_1 = matrix_1.reshape(matrix_1.shape[0],-1)
matrix_1.shape

(7115, 90)

# Converting the data into pytorch tenors

In [9]:
features = torch.from_numpy(matrix_1)
targets = torch.from_numpy(target_pagerank)
features.shape, targets.shape

(torch.Size([7115, 90]), torch.Size([7115, 1]))

In [10]:
# Activating the automatic gradient 
features.requires_grad_(True)
targets.requires_grad_(True)

tensor([[1.0270],
        [0.2524],
        [0.2524],
        ...,
        [3.7680],
        [0.4324],
        [1.7777]], dtype=torch.float64, requires_grad=True)

In [11]:
# Shuffling and dividing the indecies
n_samples = features.shape[0]
n_test = 2115
shuffled_ind = torch.randperm(n_samples)
train_ind = shuffled_ind[:-n_test]
test_ind = shuffled_ind[-n_test:]
# Dividing features and targets into tain and test sets
train_features = features[train_ind]
test_features = features[test_ind]
train_targets = targets[train_ind]
test_targets = targets[test_ind]
train_features.shape, test_features.shape, train_targets.shape, test_targets.shape

(torch.Size([5000, 90]),
 torch.Size([2115, 90]),
 torch.Size([5000, 1]),
 torch.Size([2115, 1]))

## A function for dividing train data into batches

In [12]:
# dividing train_features and train_targets into batches
def next_batch(train_features, train_targets, batch_size=100):
    num_features = train_features.shape[0]
    # Shuffling
    shuffled_ind = torch.randperm(num_features)
    shuffled_train_features = train_features[shuffled_ind]
    shuffled_train_targets = train_targets[shuffled_ind]
    # Dividing
    i = 0
    while i < num_features:
        i += batch_size
        yield (shuffled_train_features[i-batch_size:i], shuffled_train_targets[i-batch_size:i])
    return

## The feedforward neural network model

In [13]:
# The Feedforward Neural Network 
class FFNN_model(nn.Module):
    def __init__(self):
        super().__init__()
        num_features = features.shape[1]
        self.fc1 = nn.Linear(num_features, 400)
        self.fc2 = nn.Linear(400, 800)
        self.fc3 = nn.Linear(800, 200)
        self.fc4 = nn.Linear(200, 64)
        self.fc5 = nn.Linear(64, 8)
        self.fc6 = nn.Linear(8, 1)
        self.dropout1 = nn.Dropout(0.4)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.5)
    
    def forward(self, X):
        X = torch.tanh(self.fc1(X))
        X = torch.relu(self.fc2(X))
        X = self.dropout1(X)
        X = torch.relu(self.fc3(X))
        X = self.dropout3(X)
        X = torch.relu(self.fc4(X))
        X = self.dropout2(X)
        X = torch.tanh(self.fc5(X))
        return self.fc6(X)

In [14]:
# Instantiation of the model
torch.manual_seed(42)
model = FFNN_model()
model

FFNN_model(
  (fc1): Linear(in_features=90, out_features=400, bias=True)
  (fc2): Linear(in_features=400, out_features=800, bias=True)
  (fc3): Linear(in_features=800, out_features=200, bias=True)
  (fc4): Linear(in_features=200, out_features=64, bias=True)
  (fc5): Linear(in_features=64, out_features=8, bias=True)
  (fc6): Linear(in_features=8, out_features=1, bias=True)
  (dropout1): Dropout(p=0.4, inplace=False)
  (dropout2): Dropout(p=0.3, inplace=False)
  (dropout3): Dropout(p=0.5, inplace=False)
)

In [15]:
# Number of parameters
num_para = 0
for param in model.parameters():
    print(param.numel())
    num_para += param.numel()
print("----------------------")
print(f'Number of all parameters: \n{num_para}')

36000
400
320000
800
160000
200
12800
64
512
8
8
1
----------------------
Number of all parameters: 
530793


In [16]:
# Objective function and optimmizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

## Defining the Training Loop

In [17]:
def training_loop(n_epochs=1000,
                  batch_size=100,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets):
    num_features = train_features.shape[0]
    start_time = time.time()
    all_train_loss, all_test_loss = np.zeros(n_epochs), np.zeros(n_epochs)
    for epoch in range(1, n_epochs +1):
        # Training: 
        epoch_losses = []
        # looping through batches
        for train_features, train_targets in next_batch(train_features=train_features, 
                                                        train_targets=train_targets, batch_size=batch_size): 
            train_preds = model(train_features.float())
            train_loss = loss_fn(train_targets.float(), train_preds.float())
            optimizer.zero_grad()
            train_loss.backward(retain_graph=True)
            optimizer.step()
            
            epoch_losses.append(train_loss.item())
        average_epoch_loss = sum(epoch_losses)/len(epoch_losses)
        
        # Test:
        with torch.no_grad():
            test_preds = model(test_features.float())
            test_loss = loss_fn(test_targets.float(), test_preds.float())
        
        all_train_loss[epoch - 1] = average_epoch_loss
        all_test_loss[epoch - 1] = test_loss.item()
        # Printing the result: 
        if epoch == 1 or epoch % 100 == 0:
            print(f"EPOCH: {epoch:{7}}")
            print(f"MEAN TRAIN LOSS:   {average_epoch_loss:.11f},    TEST LOSS:   {test_loss.item():.11f}")
            print("-----------------------------------------")
    print("The total time = ", np.round(time.time() - start_time, 3), " seconds!")
    return all_train_loss, all_test_loss

# Training the model

In [18]:
losses = training_loop(n_epochs=2000,
                  batch_size=400,
                  optimizer=optimizer, 
                  model=model, 
                  loss_fn=criterion, 
                  train_features=train_features, 
                  test_features=test_features, 
                  train_targets=train_targets, 
                  test_targets=test_targets)

EPOCH:       1
MEAN TRAIN LOSS:   1.24386764490,    TEST LOSS:   0.96985769272
-----------------------------------------
EPOCH:     100
MEAN TRAIN LOSS:   0.69111418724,    TEST LOSS:   0.76334065199
-----------------------------------------
EPOCH:     200
MEAN TRAIN LOSS:   0.56819295883,    TEST LOSS:   0.68902349472
-----------------------------------------
EPOCH:     300
MEAN TRAIN LOSS:   0.47736743093,    TEST LOSS:   0.63417142630
-----------------------------------------
EPOCH:     400
MEAN TRAIN LOSS:   0.41414558887,    TEST LOSS:   0.59118366241
-----------------------------------------
EPOCH:     500
MEAN TRAIN LOSS:   0.35400515795,    TEST LOSS:   0.54270321131
-----------------------------------------
EPOCH:     600
MEAN TRAIN LOSS:   0.30395203829,    TEST LOSS:   0.52770364285
-----------------------------------------
EPOCH:     700
MEAN TRAIN LOSS:   0.26886439323,    TEST LOSS:   0.51323086023
-----------------------------------------
EPOCH:     800
MEAN TRAIN LOSS: 

In [19]:
model.eval()
errors = []
for i in range(n_test):
    targ = test_targets[i].item()
    feat = test_features[i].float().view(1,1,features.shape[1])
    pred = model(feat).item()
    inaccuracy = abs(1 - pred/targ) * 100
    errors.append(inaccuracy)
    if i%100 == 0:
        print(f"target {targ},    prediction: {pred}\nindex {i}:       inaccuracy: {np.round(inaccuracy, 3)}%")
        print("-----------------------------------------------------------------")

target 1.0821719086178425,    prediction: 2.162139415740967
index 0:       inaccuracy: 99.796%
-----------------------------------------------------------------
target 1.5772651444281889,    prediction: 1.9420082569122314
index 100:       inaccuracy: 23.125%
-----------------------------------------------------------------
target 0.2524391172931507,    prediction: 0.25150251388549805
index 200:       inaccuracy: 0.371%
-----------------------------------------------------------------
target 0.2524391172931507,    prediction: 0.25150251388549805
index 300:       inaccuracy: 0.371%
-----------------------------------------------------------------
target 1.9796685229065305,    prediction: 1.4952713251113892
index 400:       inaccuracy: 24.469%
-----------------------------------------------------------------
target 0.7604724388019622,    prediction: 0.9141048789024353
index 500:       inaccuracy: 20.202%
-----------------------------------------------------------------
target 0.4592662369

In [20]:
print("Average inaccuracy: ", np.round(sum(errors)/len(errors), 3))

Average inaccuracy:  13.87


In [21]:
torch.save(model, "trained_FFNN_pagerank_wiki_vote_discounted.pt")